this notebook was inspired by https://github.com/shinathan/polygon.io-stock-database/blob/main/02_market_hours.ipynb
but AFAIU it is not needed because i found [pandas_market_calendars](https://github.com/rsheftel/pandas_market_calendars?tab=readme-ov-file)

the idea was to download SPY minute bars and assume that since SPY has a lot of volume then it's possible to identify open/close hours for the market by looking at when there is a first trade for SPY.

but the first thing that i noticed was that when i executed the code in https://github.com/shinathan/polygon.io-stock-database/blob/main/02_market_hours.ipynb i got not the same results as the author had. next i checked that the hour of the first trade for SPY is not always 4am as it was assumed by the author.

the result is that AFAIU it's not a good idea to try to identify market hours from SPY.

In [30]:
%pip install polygon-api-client pandas python-dateutil


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# data was loaded by download_bars.py

# Read the CSV file
df = pd.read_csv('../../polygon-data/bars/1minute/SPY.csv')
df

,timestamp,open,high,low,close,volume,vwap,transactions,otc
0,2003-09-10 08:03:00,102.8600,102.86,102.8600,102.8600,1000.0,102.8600,1,NaN
1,2003-09-10 08:05:00,102.8600,102.86,102.8600,102.8600,500.0,102.8600,1,NaN
2,2003-09-10 08:06:00,102.8800,102.88,102.8800,102.8800,1000.0,102.8800,2,NaN
3,2003-09-10 08:09:00,102.9000,102.90,102.9000,102.9000,100.0,102.9000,1,NaN
4,2003-09-10 08:11:00,102.9000,102.90,102.9000,102.9000,300.0,102.9000,1,NaN
...,...,...,...,...,...,...,...,...,...
3961812,2025-06-13 19:55:00,597.4500,597.47,597.4500,597.4595,1826.0,597.4668,9,NaN
3961813,2025-06-13 19:56:00,597.4695,597.47,597.4695,597.4700,1305.0,597.4687,14,NaN
3961814,2025-06-13 19:57:00,597.4700,597.50,597.4700,597.5000,2343.0,597.4891,28,NaN
3961815,2025-06-13 19:58:00,597.4800,597.50,597.4400,597.4500,2670.0,597.4803,25,NaN


In [3]:
# get only the timestamp column
timestamps = pd.to_datetime(df['timestamp'])
# truncate hours
hours = timestamps.dt.floor('H')
# remove duplicate hours
unique_hours = hours.drop_duplicates()
unique_hours

/tmp/ipykernel_72803/3580168506.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours = timestamps.dt.floor('H')


0         2003-09-10 08:00:00
23        2003-09-10 09:00:00
72        2003-09-10 10:00:00
132       2003-09-10 11:00:00
192       2003-09-10 12:00:00
                  ...        
3961571   2025-06-13 15:00:00
3961631   2025-06-13 16:00:00
3961691   2025-06-13 17:00:00
3961733   2025-06-13 18:00:00
3961773   2025-06-13 19:00:00
Name: timestamp, Length: 84512, dtype: datetime64[ns]

In [4]:
# for each day get the first and last hour
first_hours = unique_hours.groupby(unique_hours.dt.date).first()
last_hours = unique_hours.groupby(unique_hours.dt.date).last()
# create a DataFrame with the first and last hours
first_last_hours = pd.DataFrame({
    'first_hour': first_hours,
    'last_hour': last_hours
})
first_last_hours

,first_hour,last_hour
timestamp,,
2003-09-10,2003-09-10 08:00:00,2003-09-10 19:00:00
2003-09-11,2003-09-11 08:00:00,2003-09-11 19:00:00
2003-09-12,2003-09-12 08:00:00,2003-09-12 18:00:00
2003-09-15,2003-09-15 08:00:00,2003-09-15 18:00:00
2003-09-16,2003-09-16 08:00:00,2003-09-16 19:00:00
...,...,...
2025-06-09,2025-06-09 04:00:00,2025-06-09 19:00:00
2025-06-10,2025-06-10 04:00:00,2025-06-10 19:00:00
2025-06-11,2025-06-11 04:00:00,2025-06-11 19:00:00


In [5]:
# rows that do not start at 04:00 or 08:00
irregular_opens = first_last_hours[
    (first_last_hours['first_hour'].dt.hour != 4) &
    (first_last_hours['first_hour'].dt.hour != 8)
]
irregular_opens

,first_hour,last_hour
timestamp,,
2004-05-21,2004-05-21 09:00:00,2004-05-21 19:00:00
2005-04-11,2005-04-11 07:00:00,2005-04-11 18:00:00
2005-04-12,2005-04-12 07:00:00,2005-04-12 19:00:00
2005-04-13,2005-04-13 06:00:00,2005-04-13 19:00:00
2005-04-14,2005-04-14 07:00:00,2005-04-14 19:00:00
...,...,...
2014-06-16,2014-06-16 06:00:00,2014-06-16 19:00:00
2014-10-27,2014-10-27 05:00:00,2014-10-27 19:00:00
2014-12-26,2014-12-26 05:00:00,2014-12-26 19:00:00


In [29]:
%pip install pandas_market_calendars


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas_market_calendars as mcal

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Show available calendars
# print(mcal.get_calendar_names())
nyse.schedule(start_date='2004-05-20', end_date='2004-05-22')

,market_open,market_close
2004-05-20,2004-05-20 13:30:00+00:00,2004-05-20 20:00:00+00:00
2004-05-21,2004-05-21 13:30:00+00:00,2004-05-21 20:00:00+00:00


In [8]:
irregular_opens

,first_hour,last_hour
timestamp,,
2004-05-21,2004-05-21 09:00:00,2004-05-21 19:00:00
2005-04-11,2005-04-11 07:00:00,2005-04-11 18:00:00
2005-04-12,2005-04-12 07:00:00,2005-04-12 19:00:00
2005-04-13,2005-04-13 06:00:00,2005-04-13 19:00:00
2005-04-14,2005-04-14 07:00:00,2005-04-14 19:00:00
...,...,...
2014-06-16,2014-06-16 06:00:00,2014-06-16 19:00:00
2014-10-27,2014-10-27 05:00:00,2014-10-27 19:00:00
2014-12-26,2014-12-26 05:00:00,2014-12-26 19:00:00


In [9]:
irregular_closes = first_last_hours[(first_last_hours['last_hour'].dt.hour != 19) &
    (first_last_hours['last_hour'].dt.hour != 20) &
    (first_last_hours['last_hour'].dt.hour != 18)]
irregular_closes

,first_hour,last_hour
timestamp,,
2003-11-28,2003-11-28 08:00:00,2003-11-28 14:00:00
2003-12-24,2003-12-24 08:00:00,2003-12-24 13:00:00
2003-12-26,2003-12-26 08:00:00,2003-12-26 13:00:00
2004-11-15,2004-11-15 08:00:00,2004-11-15 17:00:00
2004-11-26,2004-11-26 08:00:00,2004-11-26 13:00:00
2004-12-31,2004-12-31 08:00:00,2004-12-31 17:00:00
2005-04-22,2005-04-22 06:00:00,2005-04-22 17:00:00
2005-05-20,2005-05-20 07:00:00,2005-05-20 17:00:00
2005-11-25,2005-11-25 06:00:00,2005-11-25 13:00:00


In [10]:
nyse.schedule(start_date='2024-12-23', end_date='2024-12-30')

,market_open,market_close
2024-12-23,2024-12-23 14:30:00+00:00,2024-12-23 21:00:00+00:00
2024-12-24,2024-12-24 14:30:00+00:00,2024-12-24 18:00:00+00:00
2024-12-26,2024-12-26 14:30:00+00:00,2024-12-26 21:00:00+00:00
2024-12-27,2024-12-27 14:30:00+00:00,2024-12-27 21:00:00+00:00
2024-12-30,2024-12-30 14:30:00+00:00,2024-12-30 21:00:00+00:00


one more check - that SPY has at least trades for all days where pandas_market_calendars says that it was a trading day.

In [11]:
# get days from unique_hours
unique_days = unique_hours.dt.date.drop_duplicates()
first_day = str(unique_days.iloc[0])
last_day = str(unique_days.iloc[-1])
nyse_schedule = nyse.schedule(start_date=first_day, end_date=last_day)
nyse_schedule

,market_open,market_close
2003-09-10,2003-09-10 13:30:00+00:00,2003-09-10 20:00:00+00:00
2003-09-11,2003-09-11 13:30:00+00:00,2003-09-11 20:00:00+00:00
2003-09-12,2003-09-12 13:30:00+00:00,2003-09-12 20:00:00+00:00
2003-09-15,2003-09-15 13:30:00+00:00,2003-09-15 20:00:00+00:00
2003-09-16,2003-09-16 13:30:00+00:00,2003-09-16 20:00:00+00:00
...,...,...
2025-06-09,2025-06-09 13:30:00+00:00,2025-06-09 20:00:00+00:00
2025-06-10,2025-06-10 13:30:00+00:00,2025-06-10 20:00:00+00:00
2025-06-11,2025-06-11 13:30:00+00:00,2025-06-11 20:00:00+00:00
2025-06-12,2025-06-12 13:30:00+00:00,2025-06-12 20:00:00+00:00


In [12]:
# convert the index to a Series of dates to be comparable with unique_days
nyse_days = pd.Series(nyse_schedule.index.date)

In [16]:
# check that unique_days are in the NYSE schedule
print(nyse_days.isin(unique_days).all())
# and that the NYSE schedule is in unique_days
print(unique_days.isin(nyse_days).all())

True
True


In [28]:
extended = nyse.schedule(start_date=first_day, end_date=last_day, start="pre", end="post")
extended


,pre,market_open,market_close,post
2003-09-10,2003-09-10 08:00:00+00:00,2003-09-10 13:30:00+00:00,2003-09-10 20:00:00+00:00,2003-09-11 00:00:00+00:00
2003-09-11,2003-09-11 08:00:00+00:00,2003-09-11 13:30:00+00:00,2003-09-11 20:00:00+00:00,2003-09-12 00:00:00+00:00
2003-09-12,2003-09-12 08:00:00+00:00,2003-09-12 13:30:00+00:00,2003-09-12 20:00:00+00:00,2003-09-13 00:00:00+00:00
2003-09-15,2003-09-15 08:00:00+00:00,2003-09-15 13:30:00+00:00,2003-09-15 20:00:00+00:00,2003-09-16 00:00:00+00:00
2003-09-16,2003-09-16 08:00:00+00:00,2003-09-16 13:30:00+00:00,2003-09-16 20:00:00+00:00,2003-09-17 00:00:00+00:00
...,...,...,...,...
2025-06-09,2025-06-09 08:00:00+00:00,2025-06-09 13:30:00+00:00,2025-06-09 20:00:00+00:00,2025-06-10 00:00:00+00:00
2025-06-10,2025-06-10 08:00:00+00:00,2025-06-10 13:30:00+00:00,2025-06-10 20:00:00+00:00,2025-06-11 00:00:00+00:00
2025-06-11,2025-06-11 08:00:00+00:00,2025-06-11 13:30:00+00:00,2025-06-11 20:00:00+00:00,2025-06-12 00:00:00+00:00
2025-06-12,2025-06-12 08:00:00+00:00,2025-06-12 13:30:00+00:00,2025-06-12 20:00:00+00:00,2025-06-13 00:00:00+00:00
